In [1]:
is_training = True

In [2]:
import json
import shutil
import urllib

#from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import LSTM
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
from torch_geometric_temporal.nn.recurrent import GConvLSTM
from torch_geometric_temporal.signal import temporal_signal_split

from evaluation import Evaluation
from gconvlstm_lstm_model import LSTMSequenceModel
from data_loder import DatasetLoader

# **Training**

## Functions for training

In [3]:
def new_stations(): 
  ## Write the new stations! The unnecessary stations dont need delete.

    # Data to be written
    node_id = {
      "2275": "Szeged",
      "1516": "Vásárosnemény",
      "1719": "Tokaj",
      "2543": "Tiszadorogma",
      "2272": "Mindszent",
      "2274": "Algyő",
      "2756": "Gyoma",
      "2278": "Makó",
      "1722": "Tiszapalkonya",
      "2753": "Békés",
      "2271": "Csongrád",
      "210888": "Zenta"
    }

    # Serializing json
    json_object = json.dumps(node_id, indent=12)

    # Writing to parameters.json
    with open("node_id.json", "w") as outfile:
        outfile.write(json_object)

In [4]:
def load_node_id(file_name:str):
    with open(file_name, 'r') as openfile:
            # Reading from json file
        node_id = json.load(openfile)

    return node_id

In [5]:
def load_data(file_name:str,
              data_slice: tuple = (1, 501)):
    arr = np.loadtxt(file_name,  
                  usecols = (np.r_[1:13]), 
                  # np.r_ generates an array of indices
                  delimiter=",")

    place_code = arr[0,:].astype(int)

    start, end = data_slice
    value = arr[start:end,:] 

    date = np.loadtxt(file_name,
                  dtype='str',
                  usecols = 0,
                  skiprows=1,
                  delimiter=",")
    
    actual_dates = date[start-1:end-1]

    return value, place_code, actual_dates 

In [6]:
def preprocessing(values, train_ratio: float):
    train_length= int(train_ratio * len(values))  

    train = values[0:train_length]

    std = np.std(train, axis=0)
    mean = np.mean(train, axis=0)

    return std, mean

In [7]:
def get_params(value: np.array, 
              train_rat: float, 
              place_code: np.array, 
              id: dict, 
              save_name: str):

    std, mean = preprocessing(values=value, train_ratio= train_rat)
    d = dict(enumerate(place_code.flatten()))

    data_dict={}
    key_list = list(id.keys())
    val_list = list(id.values())

    for key, val in d.items():
        position = key_list.index(str(val))
        data_dict[val_list[position]]={"column_idx": key, "std": std[int(key)], "mean": mean[int(key)]}


    json_object = json.dumps(data_dict, indent=12)

    # Writing to parameters.json
    with open(save_name + ".json", "w") as outfile:   
        outfile.write(json_object)


## Model training

In [9]:
new_stations()

In [10]:
%%capture
if is_training:

    data_n = "data.csv"     # the data file name
    params_n = 'data_dict5'  # new name the parameters file
    model_n = 'Model5'       # name the new model
    past = 5              # how many days from the past are used 
    hidden_s = 50
    distance_k = 2
    
    target_l = 7
    split_ratio =  0.883       # ratio of train to data
    d_slice = (1, 22001)       # start >= 1 !!!

    value, place_code, act_dates = load_data(file_name=data_n, 
                                      data_slice=d_slice)

    node_id = load_node_id("node_id.json")

    get_params(value=value, 
            train_rat=split_ratio, 
            place_code=place_code, 
            id=node_id, 
            save_name=params_n)

    loader = DatasetLoader(data=data_n,                # data.csv read from workspace folder
                         data_params=params_n+'.json',
                         data_slice=d_slice) 

    dataset = loader.get_dataset(lags=past, target=target_l) # dataset is a StaticGraphTemporalSignal object

    train_dataset, test_dataset = temporal_signal_split(data_iterator=dataset, 
                                                      train_ratio=split_ratio)

In [11]:
if is_training:
    try:
      from tqdm import tqdm
    except ImportError:
      def tqdm(iterable):
          return iterable


    model = LSTMSequenceModel(dropout=0,
                            hidden_size=hidden_s,
                            k=distance_k,
                            model_name=model_n,
                            node_features=past,
                            target_len=target_l,
                            data_params=params_n+'.json')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


    model.train()

    all_mse = torch.tensor([])

    for epoch in tqdm(range(200)): 
        cost = 0
        h, c = None, None
        for time, snapshot in enumerate(train_dataset):
        # train_dataset is a StaticGraphTemporalSignal
        # Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=y)
        #          x: torch.Tensor with shape number of stations*node_feature  
        # edge_index: torch.Tensor with shape MassigePassing   
        #  edge_attr: weight of edges
        #          y: the target lable for each station

        ##  edge_index and edge_attr are same for every time, because the graph is Static
            snapshot = snapshot.to(device)
            y_hat, idx_Szeged = model(x=snapshot.x, 
                                    edge_index=snapshot.edge_index, 
                                    edge_weight=snapshot.edge_attr, 
                                    h=h, 
                                    c=c)
            y_hat = y_hat.to(device)
            cost = cost + (((y_hat.T-snapshot.y[idx_Szeged, :])**2).sum()/target_l)
        
        cost = cost / (time+1)

        mse = torch.tensor([cost])
        all_mse = torch.cat((all_mse, mse))

        cost.backward()
        optimizer.step()
        optimizer.zero_grad()

    model.eval() 

    # Save to file    
    torch.save(model.state_dict(), model_n)

    # Save MSE
    np.savetxt(fname=model_n + "_MSE.csv", delimiter=",", X=all_mse)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [10:01<00:00,  3.01s/it]
